In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output

import numpy as np
import pandas as pd # Used for displaying matrices
from nltk import ngrams # Used for getting bigrams
import math # Used for ceil

In [2]:
def pf_get_alphabet():
    return ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

# Convert key to uppercase and remove unwanted characters
def pf_key_convert(key):
    alphabet = pf_get_alphabet()
    key = key.upper()
    key = ''.join(list(dict.fromkeys(key)))
    for ch in key:
        if ch == 'J':
            key = key.replace(ch, 'I')
        elif ch not in alphabet:
            key = key.replace(ch, '')
    return key

# Key to matrix
def pf_key_to_matrix(key):
    key = pf_key_convert(key)
    alphabet = pf_get_alphabet()
    matrixList = []
    for ch in key:
        alphabet.remove(ch)
        matrixList.append(ch)
        key.replace(ch, ' ')
        
    matrixList = matrixList + alphabet

    matrix = []

    for i in range(0, 5):
        matrix.append([])
        for j in range(0, 5):
            matrix[i].append(matrixList[i*5+j])
        
    # print(matrixList)
    df = pd.DataFrame(matrix)
    # df.style.hide("index")
    df.style.hide(axis='columns')
    
    return df

# Convert message ready to cipher
# Remove unwanted characters
# If len is not divisable by 2 add 'X' in the end
def pf_message_convert(message):
    message = message.upper()
    alphabet = pf_get_alphabet()
    
    if 'J' in message:
        message = message.replace('J', 'I')
    
    for ch in message:
        if ch not in alphabet:
            message = message.replace(ch, '')
            
    i = 0
    while i < len(message) - 1:
        if message[i] == message[i+1]:
            message = message[:i+1] + 'X' + message[i+1:len(message)]
        i += 2
        
    if len(message) % 2 != 0:
        message += 'X'
        
    return message

# Split message to bigrams
def pf_message_to_bigrams(message):
    bigrams = list(ngrams(message, n=2))
    for i in range(1, int(len(bigrams)/2+1), 1):
        del bigrams[i]
    
    return bigrams

# Get index of 
def pf_indexOf(myList, v):
    for i, x in enumerate(myList):
        if v in x:
            return (i, x.index(v))

# Encrypt bigrams
def pf_encrypt_bigrams(bigrams, matrix):
    matrix = matrix.values.tolist()
    cipher_bigrams = []
    for i, v in enumerate(bigrams):
        y1 = pf_indexOf(matrix, bigrams[i][0])[0]
        x1 = pf_indexOf(matrix, bigrams[i][0])[1]
        y2 = pf_indexOf(matrix, bigrams[i][1])[0]
        x2 = pf_indexOf(matrix, bigrams[i][1])[1]
        
        # print(matrix[y1][x1], matrix[y1][(x1+1)%5])
        
        if y1 == y2:
            cipher_bigrams.append((matrix[y1][(x1+1)%5], matrix[y2][(x2+1)%5]))
        elif x1 == x2:
            cipher_bigrams.append((matrix[(y1+1)%5][x1], matrix[(y2+1)%5][x2]))
        else:
            cipher_bigrams.append((matrix[y1][x2], matrix[y2][x1]))
    
    return cipher_bigrams

def pf_decrypt_bigrams(cipher_bigrams, matrix):
    matrix = matrix.values.tolist()
    bigrams = []
    for i, v in enumerate(cipher_bigrams):
        y1 = pf_indexOf(matrix, cipher_bigrams[i][0])[0]
        x1 = pf_indexOf(matrix, cipher_bigrams[i][0])[1]
        y2 = pf_indexOf(matrix, cipher_bigrams[i][1])[0]
        x2 = pf_indexOf(matrix, cipher_bigrams[i][1])[1]
        
        # print(matrix[y1][x1], matrix[y1][(x1+1)%5])
        
        if y1 == y2:
            bigrams.append((matrix[y1][(x1-1)%5], matrix[y2][(x2-1)%5]))
        elif x1 == x2:
            bigrams.append((matrix[(y1-1)%5][x1], matrix[(y2-1)%5][x2]))
        else:
            bigrams.append((matrix[y1][x2], matrix[y2][x1]))
    
    return bigrams

# Convert bigrams to message
def pf_bigrams_to_string(bigrams):
    return "".join([item for t in bigrams for item in t])

In [3]:
# Get alphabet
def vg_get_alphabet():
    return ['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ё', 'Ж', 
                'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 
                'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 
                'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я']

# Convert message
def vg_convert_message(message):
    message = message.upper()
    alphabet = vg_get_alphabet()
    for ch in message:
        if ch not in alphabet:
            message = message.replace(ch, '')
    return message
    
# Create table
def vg_create_table():
    alphabet = vg_get_alphabet()
    table = []
    table.append(alphabet)
    
    for i in range(1, len(alphabet)):
        table.append(list(alphabet[i]) + alphabet[i+1:len(alphabet)] + alphabet[:i])
    
    return table

# Create dataframe 
def vg_create_dataframe(table):
    alphabet = vg_get_alphabet()
    df = pd.DataFrame(table, columns=alphabet, index=alphabet)
    pd.set_option('display.max_columns', 33)
    return df

# Shifts key by 1 letter to the right
def vg_add_key(key):
    alphabet = vg_get_alphabet()
    new_key = ''
    for i in range(0, len(key)):
        new_key += alphabet[((alphabet.index(key[i])) + 1) % len(alphabet)]
    return new_key

# Convert key to progressive key with length equal to message
def vg_convert_key(key, message):
    alphabet = vg_get_alphabet()
    key = key.upper()
    for ch in key:
        if ch not in alphabet:
            key = key.replace(ch, '')
            
    if len(key) == 0:
        return ''
    
    new_key = ''
    if len(key) < len(message):
        new_key += key
        for i in range(1, len(message), len(key)):
            key = vg_add_key(key)
            new_key += key
    else:
        new_key = key
    
    new_key = new_key[:len(message)]
        
    return new_key


# Find row
def vg_get_row(i):
    alphabet = vg_get_alphabet()
    return list(alphabet[i]) + alphabet[i+1:len(alphabet)] + alphabet[:i]

def vg_encrypt_message(message, key):
    alphabet = vg_get_alphabet()
    
    cipher_message = ''
    for i in range(0, len(message)):
        cipher_message += alphabet[(len(alphabet) + alphabet.index(message[i]) + alphabet.index(key[i])) % len(alphabet)]
        
    return cipher_message

# Decrypt message
def vg_decrypt_message(cipher_message, key):
    alphabet = vg_get_alphabet()
    
    message = ''
    for i in range(0, len(cipher_message)):
        message += alphabet[(len(alphabet) + alphabet.index(cipher_message[i]) - alphabet.index(key[i])) % len(alphabet)]
        
    return message

In [4]:
# Toggle buttons to choose cipher
cipher = widgets.ToggleButtons(
    options=['Шифр Плейфера', 'Шифр Вижнера'],
    descriptions='Выбор шифра',
    # layout=widgets.Layout(width='100%', buttonwidth='100%'),
    layout=widgets.Layout(width='auto'),
    style={'button-width': '100%'}
    
)

In [5]:
# Key input field
key_input = widgets.Text(
    placeholder='Введите ключ',
    layout=widgets.Layout(width='100%')
)

# Message input field
message_input = widgets.Textarea(
    placeholder='Введите сообщение',
    layout=widgets.Layout(width='100%')
)

# Output field
output_text = widgets.Textarea(
    value='',
    disabled=True,
    layout=widgets.Layout(width='100%')
)

vbox_text = widgets.VBox([key_input, message_input, output_text])

In [6]:
# Button to encrypt data
encrypt_button = widgets.Button(
    description='Зашифровать',
    tooltip='Зашифровать',
    layout=widgets.Layout(width='100%')
)

# Button to decrypt data
decrypt_button = widgets.Button(
    description='Расшифровать',
    tooltip='Расшифровать',
    layout=widgets.Layout(width='100%')
)

# Button to upload file
upload_file = widgets.FileUpload(
    accept='.txt',
    multiple=True,
    layout=widgets.Layout(width='100%')
)

hbox_buttons = widgets.HBox([encrypt_button, decrypt_button, upload_file])

In [7]:
def encrypt_playfair(key, message):
    message = pf_message_convert(message)
    key = pf_key_convert(key)
    if key == '':
        return 'Ошибка! Пустой ключ.'
    if message == '':
        return 'Ошибка! Пустое сообщение.'
    
    matrix = pf_key_to_matrix(key)
    bigrams = pf_message_to_bigrams(message)   
    cipher_bigrams = pf_encrypt_bigrams(bigrams, matrix)
    cipher_message = pf_bigrams_to_string(cipher_bigrams)
    print(cipher_message)
    return cipher_message

def decrypt_playfair(key, cipher_message):
    cipher_message = pf_message_convert(cipher_message)
    key = pf_key_convert(key)
    if key == '':
        return 'Ошибка! Пустой ключ.'
    if cipher_message == '':
        return 'Ошибка! Пустое сообщение.'
    matrix = pf_key_to_matrix(key)
    bigrams = pf_message_to_bigrams(cipher_message)   
    cipher_bigrams = pf_decrypt_bigrams(bigrams, matrix)
    message = pf_bigrams_to_string(cipher_bigrams)
    print(message)
    
    return message

In [8]:
def encrypt_vigenere(key, message):
    message = vg_convert_message(message)
    key = vg_convert_key(key, message)
    print(key)
    if key == '':
        return 'Ошибка! Пустой ключ.'
    if message == '':
        return 'Ошибка! Пустое сообщение.'
    
    cipher_message = vg_encrypt_message(message, key)
    
    return cipher_message

def decrypt_vigenere(key, cipher_message):
    cipher_message = vg_convert_message(cipher_message)
    key = vg_convert_key(key, cipher_message)
    print(key)
    if key == '':
        return 'Ошибка! Пустой ключ.'
    if cipher_message == '':
        return 'Ошибка! Пустое сообщение.'
    
    message = vg_decrypt_message(cipher_message, key)
    
    return message

In [9]:
def save_to_files(key, message, output):    
    f = open('output.txt', 'w')
    f.write(output)
    f.close()
    
def on_upload_change(change):
    f = open(upload_file.value[0].name, 'r')
    message_input.value = f.read().replace('\n', '')
    
upload_file.observe(on_upload_change, names='value')
    

In [10]:
def on_encrypt_clicked(event):
    key = key_input.value
    message = message_input.value
    
    output = 'Error!'
    if cipher.value == 'Шифр Плейфера':
        output = encrypt_playfair(key, message)
    else:
        output = encrypt_vigenere(key, message)
        
    output_text.value = output
    save_to_files(key, message, output)

def on_decrypt_clicked(event):
    key = key_input.value
    message = message_input.value
    
    output = 'Error!'
    if cipher.value == 'Шифр Плейфера':
        output = decrypt_playfair(key, message)
    else:
        output = decrypt_vigenere(key, message)
        
    output_text.value = output
    save_to_files(key, message, output)


encrypt_button.on_click(on_encrypt_clicked)
decrypt_button.on_click(on_decrypt_clicked)

In [11]:
layout = widgets.AppLayout(
    header=cipher,
    center=vbox_text,
    footer=hbox_buttons,
    width='400px',
    margin='20px 20px 20px 20px'
)

In [12]:
display(layout)

AppLayout(children=(ToggleButtons(layout=Layout(grid_area='header', width='auto'), options=('Шифр Плейфера', '…

In [82]:
# display(pf_key_to_matrix(key_input.value))

In [83]:
# message = pf_message_convert(message_input.value)
# print(pf_message_to_bigrams(message))


In [88]:
# message = vg_convert_message(message_input.value)
# print(vg_convert_key(key_input.value, message))